In [1]:
from os.path import join

main_dir = "/content/drive/MyDrive/Fire_Mapping_Project/"

In [2]:
model_dir = join(main_dir,"model")
data_dir = join(main_dir, "data")

print("model_dir", model_dir)
print("data_dir", data_dir)

model_dir /content/drive/MyDrive/Fire_Mapping_Project/model
data_dir /content/drive/MyDrive/Fire_Mapping_Project/data


In [3]:
cd $model_dir

/content/drive/MyDrive/Fire_Mapping_Project/model


In [4]:
import argparse
import os
import random

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch import FloatTensor
# import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms

from net.unet import U_Net
from net.attention_unet import AttU_Net
from dataset import FireDataset

def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)

def get_dataset(img_dir, mask_dir, transform_mode='basic'):
    return FireDataset(img_dir, mask_dir, transform_mode)

# +
def evaluate(args, model, test_dataloader):
    model.eval()
    criterion = nn.BCELoss()
    eval_losses = []
    sensitivities = []
    specificities = []
    ppvs = []
    npvs = []
    f1s = []

    for step, (images, masks) in enumerate(test_dataloader):

        if args.cuda:
            images = images.cuda(args.device_id)
            masks = masks.cuda(args.device_id)

        with torch.no_grad():
            output_masks = model(images)
            output_masks = torch.sigmoid(output_masks)
            loss = criterion(output_masks, masks)
            eval_losses.append(loss.item())
            
            output_masks = output_masks.detach().cpu().numpy()
            output_masks = np.where(output_masks > args.cutoff, 1, 0)
            masks = masks.detach().cpu().numpy()
            
            #Scoring

            tp = ((output_masks == 1) & (masks == 1)).sum()
            tn = ((output_masks == 0) & (masks == 0)).sum()
            fp = ((output_masks == 1) & (masks == 0)).sum()
            fn = ((output_masks == 0) & (masks == 1)).sum()
            
#             print("tp, tn, fp ,fn", tp, tn, fp ,fn)
            
            sensitivity = tp / (tp + fn)
            specificity = tn / (fp + tn)
            ppv = tp / (fp + tp)
            npv = tn / (fn + tn)
            f1 = 2*tp/(2*tp + fp + fn)
            
            sensitivities.append(sensitivity)
            specificities.append(specificity)
            ppvs.append(ppv)
            npvs.append(npv)
            f1s.append(f1)

#     return np.mean(eval_losses)
    return np.mean(eval_losses), np.mean(sensitivities), np.mean(specificities), np.mean(f1s), np.mean(ppvs), np.mean(npvs)


# -

def train(args, train_dataloader, test_dataloader):

    epochs = args.epochs
    batch_size = args.batch_size
    learning_rate = args.lr
    log_interval = args.log_interval

    # Model
    if args.net == 'unet':
        model = U_Net(img_ch=1, output_ch=1)
    if args.net == 'attn_unet':
        model = AttU_Net(img_ch=1, output_ch=1)

    if args.cuda:
        model.cuda(args.device_id)

    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Loss criterion - BCE: binary cross entropy loss, as 2 classes in mask
    criterion = nn.BCELoss()


    # Begin training
    print("Model :", args.net)
    print("Cutoff :", args.cutoff)
    print("Train Dataloader size :",len(train_dataloader))
    print("Transform mode :",args.transform_mode)
    print("Batch size :", batch_size)
    print("Epochs :",epochs)
    print("Begin Training")
    model.train()
    model.zero_grad()

    best_eval_f1 = 0.5

    for epoch in range(epochs):
        epoch_losses = []
        for step, (images, masks) in enumerate(train_dataloader):
            model.train()

            if args.cuda:
                images = images.cuda(args.device_id)
                masks = masks.cuda(args.device_id)

            output_masks = model(images)
            output_masks = torch.sigmoid(output_masks)
            loss = criterion(output_masks, masks)
            # print(loss)

            loss.backward()
            optimizer.step()
            model.zero_grad()

            epoch_losses.append(loss.item())

        # Printing
        if epoch % log_interval == 0:
            current_epoch_loss = np.mean(epoch_losses)
            eval_loss, eval_sens, eval_spec, eval_f1, eval_ppv, eval_npv = evaluate(args, model, test_dataloader)
            print("Epoch %d, Loss %.3f, Eval: Loss %.3f, Sens %.3f, Spec %.3f, F1 %.3f, PPV %.3f, NPV %.3f" \
                  %(epoch, current_epoch_loss, eval_loss, eval_sens, eval_spec, eval_f1, eval_ppv, eval_npv))

            #Save the model with minimun evaluation sensitivity :
            if epoch > 10 and eval_sens != 1.0 and eval_f1 > best_eval_f1:
                # Save the model
                print("Saving epoch %d model"%(epoch))
                torch.save(model.state_dict(), 'trained_model_'+str(epoch)+'_'+str(args.net))
                best_eval_f1 = eval_f1


In [5]:
class Args:
  def __init__(self):
    self.train_img_dir = join(data_dir, "train_images")
    self.train_mask_dir = join(data_dir, "train_masks")
    self.test_img_dir = join(data_dir, "test_images")
    self.test_mask_dir = join(data_dir, "test_masks")

    self.net = 'attn_unet'
    self.epochs = 100
    self.batch_size = 8
    self.cutoff = 0.30
    self.lr = 0.0001
    self.log_interval = 1
    self.transform_mode = 'crop_hflip_vflip'
    self.device_id = 0
    self.seed = 10


In [6]:
args = Args()
print(vars(args))
print(args.batch_size)

{'train_img_dir': '/content/drive/MyDrive/Fire_Mapping_Project/data/train_images', 'train_mask_dir': '/content/drive/MyDrive/Fire_Mapping_Project/data/train_masks', 'test_img_dir': '/content/drive/MyDrive/Fire_Mapping_Project/data/test_images', 'test_mask_dir': '/content/drive/MyDrive/Fire_Mapping_Project/data/test_masks', 'net': 'attn_unet', 'epochs': 100, 'batch_size': 8, 'cutoff': 0.3, 'lr': 0.0001, 'log_interval': 1, 'transform_mode': 'crop_hflip_vflip', 'device_id': 0, 'seed': 10}
8


In [ ]:
# Set CUDA
args.cuda = True if torch.cuda.is_available() else False
print("Cuda",args.cuda)

# Set seed
set_seed(args)

# Get datasets
train_dataset = get_dataset(args.train_img_dir, args.train_mask_dir, args.transform_mode)
test_dataset = get_dataset(args.test_img_dir, args.test_mask_dir, args.transform_mode)

train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)

train(args, train_dataloader, test_dataloader)

Cuda True
Model : attn_unet
Cutoff : 0.3
Train Dataloader size : 4
Transform mode : crop_hflip_vflip
Batch size : 8
Epochs : 100
Begin Training


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in long_scalars


Epoch 0, Loss 0.598, Eval: Loss 0.647, Sens 1.000, Spec 0.000, F1 0.144, PPV 0.085, NPV nan
Epoch 1, Loss 0.514, Eval: Loss 0.631, Sens 1.000, Spec 0.000, F1 0.135, PPV 0.079, NPV nan
Epoch 2, Loss 0.463, Eval: Loss 0.610, Sens 1.000, Spec 0.000, F1 0.132, PPV 0.077, NPV nan
Epoch 3, Loss 0.423, Eval: Loss 0.588, Sens 1.000, Spec 0.000, F1 0.154, PPV 0.094, NPV nan
Epoch 4, Loss 0.410, Eval: Loss 0.557, Sens 1.000, Spec 0.000, F1 0.137, PPV 0.080, NPV nan
Epoch 5, Loss 0.390, Eval: Loss 0.523, Sens 1.000, Spec 0.000, F1 0.155, PPV 0.095, NPV nan
Epoch 6, Loss 0.376, Eval: Loss 0.472, Sens 0.347, Spec 0.182, F1 0.091, PPV 0.058, NPV 0.722
Epoch 7, Loss 0.371, Eval: Loss 0.441, Sens 0.063, Spec 0.610, F1 0.046, PPV 0.037, NPV 0.856
